Because image chips of size 25x256 appear to small to segment fields that can rang efrom 8x8 to 256x256 pixels, we need a variable gridding program to see how input size effects output model accuracy.

In [ ]:
from affine import Affine
import mercantile

import rasterio
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT
import os

# change your root directory
ROOT_DIR = "/home/rave/tana-crunch/waves/cropmask/data/raw/wv2/"
# create folders below ahead of time
os.chdir(ROOT_DIR)
WV2_IMAGERY_DIR = 'projectedtiffs/'
LABELS_DIR = 'rasterized_wv2_labels'
OUT_WV2_DIR = 'adjusted_gridded_tifs'
OUT_LABELS_DIR = 'adjusted_gridded_labels'


In [ ]:
import random
random.randint(100000000,999999999)

In [ ]:
img_list = sorted(next(os.walk(WV2_IMAGERY_DIR))[2])
off_list = [img for img in img_list if 'OS' in img]
gs_list = [img for img in img_list if 'GS' in img]
label_list = sorted(next(os.walk(LABELS_DIR))[2])

In [ ]:
import os, gdal

for off, gs, label in zip(off_list, gs_list, label_list):
    off_path = os.path.join(WV2_IMAGERY_DIR,off)
    gs_path = os.path.join(WV2_IMAGERY_DIR,gs)
    label_path = os.path.join(LABELS_DIR,label)
    #assign unique name to each gridded tif, keeping season suffix
    #assigning int of same length as ZA0932324 naming convention

    tile_size_x = 512
    tile_size_y = 512

    ds_off = gdal.Open(off_path)
    band_off = ds_off.GetRasterBand(1)

    ds_gs = gdal.Open(gs_path)
    band_gs = ds_gs.GetRasterBand(1)
    xsize_gs = band_gs.XSize
    ysize_gs = band_gs.YSize   
    
    ds_label = gdal.Open(label_path)
    band_label = ds_label.GetRasterBand(1)


    for i in range(0, xsize_gs, tile_size_x):
        for j in range(0, ysize_gs, tile_size_y):
            unique_id = str(random.randint(100000000,999999999))
            out_path_off = os.path.join(OUT_WV2_DIR,unique_id)+ off[2:]
            out_path_gs = os.path.join(OUT_WV2_DIR,unique_id)+gs[2:]
            out_path_label = os.path.join(OUT_LABELS_DIR,unique_id)+label[2:]
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(off_path) + " " + str(out_path_off)
            os.system(com_string)
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(gs_path) + " " + str(out_path_gs)
            os.system(com_string)
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(label_path) + " " + str(out_path_label)
            os.system(com_string)